In [1]:
import os
import numpy as np
import glob
import csv
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

In [2]:
%matplotlib inline

Organizes data from scan and creates symbolic links in the /data directory of the CNI server


In [61]:
#preliminary stuff
#get subs and make mapping for sub id
data_dir = '/nimsfs/smcclure/fd/'
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'scripts/sub_scan_mappings.txt'

subs = os.listdir(data_dir)
subs = sorted(subs, key= lambda s: int(s.split('_')[2])) #sort subjects by session ID for ease of making text file

#make dictionary mapping scan ids to subject ids
scan_to_id = {}
with open(subj_file) as f:
    for line in f:
        (key,val) = line.strip().split(' ')
        scan_to_id[key] = val

#make subject folder if it doesn't exist
for scan in scan_to_id.keys():
    sub_dir = home_dir + 'data/fd_' + scan_to_id[scan]
    if not os.path.isdir(sub_dir):
        os.mkdir(sub_dir)

In [64]:
#create symbolic links for T1s
for scan in scan_to_id.keys():
    sub = scan_to_id[scan]
    
    #create new directory if it doesnt exist
    anat_dir = home_dir + 'data/fd_' + sub + '/anat'
    if not os.path.isdir(sub_dir):
        os.mkdir(anat_dir)
    
    if scan != '20150304_0853_9027': #t1 from a different scan for this subject
    
        #get list of files in directory sorted by scan #
        sub_files = os.listdir(data_dir + scan)
        sub_files = sorted(sub_files, key = lambda s: int(s.split('_')[1])) 

        #get T1 (anatomical)
        t1 = [f for f in sub_files if f.split('_')[3] == 'T1w'][0]

        #create symbolic link
        new_anat = anat_dir + '/anat'
        old_anat = data_dir + '/' + scan + '/' + t1
        cmd_str = 'ln -s ' + old_anat + ' ' + new_anat
        print cmd_str

ln -s /nimsfs/smcclure/fd//20150410_1106_9388/9388_10_1_T1w_Whole_brain_1mm /data/home/iballard/fd/data/fd_118/anat/anat
ln -s /nimsfs/smcclure/fd//20150410_0918_9387/9387_9_1_T1w_Whole_brain_1mm /data/home/iballard/fd/data/fd_117/anat/anat
ln -s /nimsfs/smcclure/fd//20150308_1902_9065/9065_10_1_T1w_Whole_brain_1mm /data/home/iballard/fd/data/fd_110/anat/anat
ln -s /nimsfs/smcclure/fd//20150305_1848_9040/9040_10_1_T1w_Whole_brain_1mm /data/home/iballard/fd/data/fd_109/anat/anat
ln -s /nimsfs/smcclure/fd//20150303_1216_9020/9020_10_1_T1w_Whole_brain_1mm /data/home/iballard/fd/data/fd_106/anat/anat
ln -s /nimsfs/smcclure/fd//20150302_1945_9017/9017_11_1_T1w_Whole_brain_1mm /data/home/iballard/fd/data/fd_105/anat/anat
ln -s /nimsfs/smcclure/fd//20150407_1921_9367/9367_10_1_T1w_Whole_brain_1mm /data/home/iballard/fd/data/fd_112/anat/anat
ln -s /nimsfs/smcclure/fd//20150301_1611_9006/9006_13_1_T1w_Whole_brain_1mm /data/home/iballard/fd/data/fd_101/anat/anat
ln -s /nimsfs/smcclure/fd//201504

In [ ]:
    ##get mux files (main runs)
    mux = [f for f in sub_files if f.split('_')[4] == '16mm']
    mux = sorted(mux, key = lambda s: int(s.split('_')[1]))
    
    ##toss aborted mux scans (shorter than 2 minutes)
    
    
    ##get calibration scans for field map unwarping
    cal2 = [f for f in sub_files if f.split('_')[4] == 'cal2']
    cal2 = sorted(cal2, key = lambda s: int(s.split('_')[1]))
    
    ##need to get just the preceding cal1 scans for field mapping (should be just 4/8)
    cal1 = []
    for cal in cal2:
        preceding_index = int(cal.split('_')[1]) - 1
        c1 = [f for f in sub_files if int(f.split('_')[1]) == preceding_index]
        cal1.append(c1)
    
    
#ln -s /nimsfs/smcclure/efarb/20110506_1157_373/ efarb_sub001